In [1]:
import ee
import geemap
import json
import os
#from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import segment_testing_my_gee_functions as mgf
import simple_fishnet as fish
import classification_loop as cloop

In [2]:
ee.Authenticate()
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [10]:
shp_folder_path = 'E:/!!Research/!!!Data/example_data/example_data_py/example_data_shp/'
#shp_file = 'example_data.shp'
#shp_file = 'ex_data_med.shp'
shp_file = 'ex_data_large.shp'
example_data = geemap.shp_to_ee(f'{shp_folder_path}{shp_file}')

In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
# checking that the imported layer loadedn
Map.addLayer(example_data)

In [ ]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir_naip = 'E:/!!Research/!!!Data/example_data/example_data_py/naip/ex_naip.tif'

geemap.ee_export_image(
    naip, filename = exp_file_dir_naip, scale=1, region=shp_geom, file_per_band=False
)

Testing pixel-based classification

In [ ]:
folder_dir = 'E:/!!Research/!!!Data/example_data/example_data_py'
data_dir = f'fishnet_classified'
shp_dir = f'{folder_dir}/ex_data_fishnet'

cloop.download_classified(folder_dir, data_dir, shp_dir)

In [ ]:
def pixel_based_classify():
    naip = mgf.naip_func(example_data)
    
    savi = mgf.naip_savi_endvi(naip)
    
    def gauss_three_func(given_image):
        ### gaussian smoothing
        gaussianKernel = ee.Kernel.gaussian(
          radius = 3, # was originally 3, also tested 10
          units = 'pixels'
        )
        
        gaussian_smooth = given_image.convolve(gaussianKernel)
        return gaussian_smooth
    
    gauss = gauss_three_func(savi)
    vis_param_smooth = {'bands' : ['R', 'G', 'B'], 
                   'min' : 0, 
                   'max' : 256,
                   'gamma' : 1}
    
    grayscale = naip.expression(
          '(0.3 * R) + (0.59 * G) + (0.11 * B)', {
          'R': naip.select(['R']),
          'G': naip.select(['G']),
          'B': naip.select(['B'])
    })
    
    # if wanting to use a grayscale image
    int_gray = grayscale.int()
    glcmTexture = int_gray.glcmTexture(5)
    
    imp_glcm_bands = glcmTexture.select(['constant_savg', 'constant_contrast'])
    
    glcm_segment = imp_glcm_bands.addBands(gauss, ['R', 'G', 'B', 'N', 'savi']) #may need to include 'savi' and 'endvi'
    
    standardized = mgf.stdrd_func(glcm_segment, example_data)
    
    # point-based algorithm
    max_clusters = 10 # tested with 10
    
    pb_classed = mgf.pb_class(example_data, standardized, max_clusters)

In [ ]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir_pb = 'E:/!!Research/!!!Data/example_data/example_data_py/classified/ex_pb_classed_10_no_savi.tif'

geemap.ee_export_image(
    pb_classed, filename = exp_file_dir_pb, scale=2, region=shp_geom, file_per_band=False
)